# Does everything to run entropy and creates slurm files to submit

Goal is 4 chains k=2-8.

**Creates mpgl file from PL in vcf if running GATK**.   


In [1]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [2]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import random
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

#### assessing with limited snps just to get code right. 

In [3]:
root = '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk'

In [4]:
cd $root

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


### Create mgpl & pntest files with associated dir   

Uses custom R script to do so

In [5]:
ent_dir =os.path.join(root,'entropy')
assert ent_dir

In [6]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/entropy


In [7]:
!gunzip 'gatk.ALL.final.vcf.gz'

In [8]:
%load_ext rpy2.ipython

In [9]:
%%R
require(readr)
require(MASS)
require(LEA)
require(ggplot2)
library(vcfR)

source('/data/gpfs/home/tfaske/g/src/R/Imports.R')

setwd('/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/entropy')

R[write to console]: Loading required package: readr

R[write to console]: Loading required package: MASS

R[write to console]: Loading required package: LEA

R[write to console]: Loading required package: ggplot2

R[write to console]: 
   *****       ***   vcfR   ***       *****
   This is vcfR 1.8.0 
     browseVignettes('vcfR') # Documentation
     citation('vcfR') # Citation
   *****       *****      *****       *****




In [10]:
%%R
vcf <- read.vcfR("gatk.ALL.final.vcf", verbose = FALSE)

#get positions
chrom <- getCHROM(vcf)
pos <- getPOS(vcf)
pos_ID <- paste(chrom,pos,sep = ':')

#get pl and gt
pl <- extract.gt(vcf, element = 'PL')
gt <- extract.gt(vcf, element = 'GT')

In [11]:
%%R
## check out PL and pos_ID
print(length(pos_ID))
print(pos_ID[1:10])
pl[1:5,1:5]

[1] 35311
 [1] "dDocent_Contig_33205:21" "dDocent_Contig_45538:49"
 [3] "dDocent_Contig_47372:37" "dDocent_Contig_53695:69"
 [5] "dDocent_Contig_54063:13" "dDocent_Contig_55085:75"
 [7] "dDocent_Contig_57409:13" "dDocent_Contig_57557:38"
 [9] "dDocent_Contig_59197:37" "dDocent_Contig_59336:19"
                        AT2_BB_1   AT2_BB_10   AT2_BB_11  AT2_BB_12 
dDocent_Contig_33205_21 "0,18,247" "0,0,209"   "0,0,408"  "0,13,526"
dDocent_Contig_45538_49 "0,3,485"  "315,0,270" "78,0,78"  "75,0,120"
dDocent_Contig_47372_37 "0,57,772" "0,36,482"  "0,12,159" "0,24,303"
dDocent_Contig_53695_69 "0,51,688" "0,35,623"  "0,36,478" "0,24,296"
dDocent_Contig_54063_13 "0,30,402" "0,36,483"  "0,24,307" "0,75,998"
                        AT2_BB_13  
dDocent_Contig_33205_21 "0,87,1189"
dDocent_Contig_45538_49 "141,0,366"
dDocent_Contig_47372_37 "0,33,459" 
dDocent_Contig_53695_69 "0,12,144" 
dDocent_Contig_54063_13 "0,99,1350"


In [12]:
%%R
#converts pl to mpgl format
mpgl <- apply(pl, c(1,2), function(df) gsub(',',' ',df,fixed=TRUE))

In [15]:
%%R
#check pl and mggl
print(pl[600:605,600:605])
print(mpgl[600:605,600:605])

                         AW4_OE_4           AW4_OE_5        AW4_OE_6        
dDocent_Contig_390288_38 "60,0,10,33,353"   "0,5,12,23,114" "0,9,21,42,292" 
dDocent_Contig_390322_20 "0,21,51,102,686"  "0,0,0,0,167"   "0,16,39,78,543"
dDocent_Contig_391987_82 "0,0,0,0,0"        "0,0,0,0,0"     "0,0,0,0,0"     
dDocent_Contig_392092_31 "0,11,27,54,376"   "68,1,0,6,110"  "0,6,15,30,197" 
dDocent_Contig_392438_18 "473,0,11,76,1206" "143,5,0,5,143" "92,0,12,43,494"
dDocent_Contig_395037_22 "0,15,36,72,501"   "0,4,9,18,113"  "0,19,45,90,615"
                         AW4_OE_7          AW4_OE_8        AW4_OE_9         
dDocent_Contig_390288_38 "0,1,3,6,42"      "0,0,0,0,0"     "0,15,36,72,489" 
dDocent_Contig_390322_20 "0,0,0,0,0"       "0,0,0,0,0"     "0,7,18,36,251"  
dDocent_Contig_391987_82 "0,0,0,0,0"       "0,0,0,0,0"     "0,0,0,0,0"      
dDocent_Contig_392092_31 "0,2,6,12,84"     "0,0,0,0,0"     "0,4,9,18,125"   
dDocent_Contig_392438_18 "58,0,13,43,434"  "0,0,0,14,334"  "0,19,45,90,627" 

In [16]:
%%R
rownames(mpgl) <- pos_ID
dim(mpgl)
write.table(mpgl,'mpgl.txt',quote=F,col.names = F) 

In [17]:
%%R
### pntest_mean ###
mean_pl <- function(GP){
    gps <- as.numeric(unlist(strsplit(as.character(GP),split = ',')))
    ploidy <- length(gps)-1
    gps <- 10 ^ (gps/-10)
    mean <- sum(gps*(0:ploidy))/sum(gps)
    return(mean) 
}

system.time(pntest_mean <- apply(pl,1:2,mean_pl))

   user  system elapsed 
413.889   5.823 419.630 


In [18]:
%%R 
rownames(pntest_mean) <- pos_ID
dim(pntest_mean)

write.table(pntest_mean,'pntest_mean.txt',quote=F)

In [22]:
%%R
#make sure ploidy makes sense
print(apply(pntest_mean,2,function(df) range(df, na.rm=TRUE)))

     AT2_BB_1 AT2_BB_10    AT2_BB_11 AT2_BB_12 AT2_BB_13 AT2_BB_14 AT2_BB_15
[1,]    1e-12     1e-12 1.000000e-12     1e-12     1e-12     1e-12     1e-12
[2,]    2e+00     2e+00 1.999999e+00     2e+00     2e+00     2e+00     2e+00
     AT2_BB_16    AT2_BB_17 AT2_BB_2     AT2_BB_3 AT2_BB_4 AT2_BB_5
[1,]     1e-12 1.000000e-12    1e-12 1.000000e-12    1e-12    1e-12
[2,]     2e+00 1.999992e+00    2e+00 1.999999e+00    2e+00    2e+00
         AT2_BB_6 AT2_BB_7 AT2_BB_8 AT2_BB_9 AT2_BC_17    AT2_BC_18 AT2_BC_19
[1,] 3.162278e-13    1e-12    1e-12    1e-12     1e-12 7.943282e-14     1e-12
[2,] 2.000000e+00    2e+00    2e+00    2e+00     2e+00 2.000000e+00     2e+00
        AT2_BC_20    AT2_BC_21    AT2_BC_22 AT2_BC_23     AT2_BT_1     AT2_BT_2
[1,] 1.584893e-14 1.000000e-12 1.000000e-12     1e-12 1.000000e-12 3.981072e-15
[2,] 2.000000e+00 1.999998e+00 1.999996e+00     2e+00 1.999996e+00 2.000000e+00
     AT2_BT_3     AT2_BT_4     AT2_BT_5 AT2_BT_57    AT2_BT_58    AT2_BT_59
[1,]    1e-12 1

         AT4_DS_3     AT4_DS_4     AT4_DS_5     AT4_DS_6     AT4_DS_7
[1,] 1.995262e-07 1.258925e-07 1.258924e-06 7.943276e-07 9.999990e-07
[2,] 3.999999e+00 3.999999e+00 3.999999e+00 3.999999e+00 3.999999e+00
         AT4_DS_8     AT4_SL_1     AT4_SL_2   AT4_SL_284   AT4_SL_285
[1,] 6.309569e-07 3.162268e-06 1.584891e-06 1.258924e-06 9.999902e-06
[2,] 3.999999e+00 3.999001e+00 3.993722e+00 3.999749e+00 3.999900e+00
       AT4_SL_286     AT4_SL_3     AT4_SL_4     AT4_SL_5     AT4_SL_6
[1,] 1.258924e-06 9.999902e-06 3.981056e-06 9.999902e-06 7.943276e-07
[2,] 3.999874e+00 3.992104e+00 3.998008e+00 3.996846e+00 3.999987e+00
         AT4_SL_7     AT4_WT_1     AT4_WT_2     AT4_WT_3     AT4_WT_4
[1,] 9.999990e-07 9.999902e-06 1.995262e-07 3.162277e-07 3.162268e-06
[2,] 3.999996e+00 3.999995e+00 3.999968e+00 3.999999e+00 3.999921e+00
         AT4_WT_5     AT4_WT_6     AT4_WT_7     AV2_AN_1 AV2_AN_2 AV2_AN_3
[1,] 7.943220e-06 7.943276e-07 1.258924e-06 7.943282e-16    1e-12    1e-12
[2,] 3.999

[1,] 3.162277e-07 3.162278e-08 1.258924e-06 1.995262e-07 1.995262e-07
[2,] 3.999999e+00 3.999999e+00 3.999960e+00 3.999987e+00 3.999999e+00
         AV4_LM_5     AV4_LM_6    AV4_LM_7     AV4_LM_8     AV4_LM_9
[1,] 1.995262e-07 9.999901e-06 5.01187e-07 6.309569e-07 3.981070e-07
[2,] 3.999999e+00 3.999801e+00 3.99990e+00 3.999994e+00 3.999998e+00
         AV4_LS_1    AV4_LS_10    AV4_LS_11    AV4_LS_12   AV4_LS_13
[1,] 6.309534e-06 1.995258e-06 3.981056e-06 6.309534e-06 7.94322e-06
[2,] 3.999921e+00 3.998742e+00 3.980315e+00 3.999994e+00 3.99999e+00
        AV4_LS_14    AV4_LS_15    AV4_LS_16    AV4_LS_17   AV4_LS_18
[1,] 6.309534e-06 9.999902e-06 9.999902e-06 1.258924e-06 2.51188e-06
[2,] 3.999987e+00 3.968867e+00 3.999001e+00 3.998742e+00 3.99998e+00
        AV4_LS_19     AV4_LS_2    AV4_LS_20    AV4_LS_21    AV4_LS_22
[1,] 2.511880e-06 6.309534e-06 6.309534e-06 9.999902e-06 3.162268e-06
[2,] 3.999992e+00 3.999900e+00 3.999874e+00 3.987505e+00 3.998742e+00
        AV4_LS_23    AV4_LS_2

[1,] 1.995262e-07 1.258925e-07 1.584891e-06 9.99999e-07 3.162277e-07
[2,] 3.999999e+00 3.999999e+00 3.999684e+00 4.00000e+00 3.999999e+00
         AW4_SR_7     AW4_SR_8     AW4_SR_9     AW4_SS_1    AW4_SS_10
[1,] 2.511886e-07 9.999999e-08 6.309569e-07 1.258924e-06 1.258924e-06
[2,] 3.999999e+00 4.000000e+00 3.999999e+00 3.999999e+00 3.999999e+00
         AW4_SS_2     AW4_SS_3     AW4_SS_4     AW4_SS_5     AW4_SS_6
[1,] 3.981070e-07 9.999990e-07 6.309534e-06 2.511886e-07 3.162268e-06
[2,] 3.999999e+00 3.999999e+00 3.999999e+00 4.000000e+00 3.999994e+00
         AW4_SS_7    AW4_SS_8     AW4_SS_9     AW4_WH_1     AW4_WH_2
[1,] 7.943276e-07 9.99999e-07 2.511886e-07 7.943276e-07 2.511886e-07
[2,] 3.999999e+00 4.00000e+00 3.999999e+00 3.999999e+00 3.999999e+00
         AW4_WH_3     AW4_WH_4     AW4_WH_5     AW4_WH_6     AW4_WJ_1
[1,] 9.999999e-08 1.995258e-06 1.258924e-06 5.011870e-07 6.309569e-07
[2,] 3.999999e+00 3.999994e+00 3.999998e+00 3.999999e+00 3.999999e+00
        AW4_WJ_10     AW4

In [20]:
%%R
#convert_gt
convert_gt <- function(g){
    gts <- as.numeric(unlist(strsplit(as.character(g),split = '/')))
    gt_sum <- sum(gts)
    return(gt_sum)
}
system.time(gt_sum <- apply(gt,1:2,convert_gt))

   user  system elapsed 
361.765   2.289 363.985 


In [21]:
%%R 
#find correlation between gt and pntest_mean
cor_list <- rep(NA,times=nrow(pntest_mean))
for (i in 1:nrow(pntest_mean)){
    cor_out <- cor(pntest_mean[i,],gt_sum[i,],use='pairwise.complete.obs')
    cor_list[i] <- cor_out
}
summary(cor_list)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
-0.2748  0.4563  0.5912  0.5745  0.7226  0.9864       3 


### Create ldak files in R

In [114]:
%%R
#create Pop_ID file 

#get IDs and check
indv <- colnames(pl)
#print(length(indv))
#print(indv[1:10])

#### make SP_POP_ID ######
Sp <- rep(NA,times=length(indv))
Ploidy <- rep(NA,times=length(indv))
Pop <- rep(NA,times=length(indv))
ID <- rep(NA,times=length(indv))
All <- rep(NA,times=length(indv))
for (i in 1:length(indv)){
  SpP <- unlist(strsplit(as.character(indv[i]),"_"))[1]
  Sp[i] <- gsub('\\d','',SpP,perl=TRUE)
  Ploidy[i] <-  gsub('(\\D)','',SpP,perl=TRUE)
  Pop[i] <- unlist(strsplit(as.character(indv[i]),"_"))[2]
  ID[i] <- unlist(strsplit(as.character(indv[i]),"_"))[3]
  All[i] <- as.character(indv[i])
}
Pop_ID <- data.frame(Sp=Sp,Ploidy=Ploidy,Pop=Pop,ID=ID,All=All,SpPloidy=paste0(Sp,Ploidy))

write.csv(Pop_ID,"Pop_ID.csv",row.names = FALSE)

Pop_ID

   Sp Ploidy Pop  ID        All SpPloidy
1  AT      2  EW  36  AT2_EW_36      AT2
2  AT      2  HC   2   AT2_HC_2      AT2
3  AT      2  VM   5   AT2_VM_5      AT2
4  AT      2  YB  79  AT2_YB_79      AT2
5  AT      4  DS   2   AT4_DS_2      AT4
6  AT      4  SL   2   AT4_SL_2      AT4
7  AT      4  WT   2   AT4_WT_2      AT4
8  AV      2  FP   6   AV2_FP_6      AV2
9  AV      2  GC   9   AV2_GC_9      AV2
10 AV      2  TF 156 AV2_TF_156      AV2
11 AV      2  UV   2   AV2_UV_2      AV2
12 AV      4  JU   5   AV4_JU_5      AV4
13 AV      4  LM   5   AV4_LM_5      AV4
14 AV      4  LS   5   AV4_LS_5      AV4
15 AV      4  RR   5   AV4_RR_5      AV4


In [119]:
%%R
Pop_ID <- read.csv("Pop_ID.csv")
pca_df <- PCA_entropy(t(pntest_mean))

    PC1     PC2     PC3     PC4     PC5 
0.23385 0.13007 0.10995 0.09503 0.09200 


R[write to console]: Error in method1$x[, 1:27] : subscript out of bounds
Calls: <Anonymous> -> <Anonymous> -> withVisible -> PCA_entropy




Error in method1$x[, 1:27] : subscript out of bounds
Calls: <Anonymous> -> <Anonymous> -> withVisible -> PCA_entropy


In [120]:
%%R
pca_plot_df <- cbind(Pop_ID,pca_df)
col3 <- c("#899DA4","#C93312","#FAEFD1")
ggplot(data = pca_df, aes(x=PC1,y=PC2,fill=Sp,shape=as.character(Ploidy))) + 
  geom_point(colour='black',size = 4)+ #ggtitle("PCA ARTR all") +
  #xlab(paste("PC",1," (",pve[1]*100,"%)",sep="")) + ylab(paste("PC",2," (",pve[2]*100,"%)",sep=""))  +
  scale_fill_manual(name='Subspecies:',values = col3) +
  scale_shape_manual(name='Ploidy:',values = c(21,24)) + 
  theme_bw() + 
  theme(#legend.position = 'none',
    axis.text = element_text(size=11), 
    axis.title = element_text(size = 13, colour="black",face = "bold",vjust = 1),
    panel.border = element_rect(size = 1.5, colour = "black"),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank())

R[write to console]: Error in cbind(Pop_ID, pca_df) : object 'pca_df' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> cbind




Error in cbind(Pop_ID, pca_df) : object 'pca_df' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> cbind


In [203]:
%%R
#######################################################################################
                                    #### LDA ####
#######################################################################################

k2<-kmeans(pca_df[,1:5],2,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k3<-kmeans(pca_df[,1:5],3,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k4<-kmeans(pca_df[,1:5],4,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k5<-kmeans(pca_df[,1:5],5,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k6<-kmeans(pca_df[,1:5],6,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k7<-kmeans(pca_df[,1:5],7,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k8<-kmeans(pca_df[,1:5],8,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k9<-kmeans(pca_df[,1:5],9,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k10<-kmeans(pca_df[,1:5],10,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k11<-kmeans(pca_df[,1:5],11,iter.max=10,nstart=10,algorithm="Hartigan-Wong")

ldak2<-lda(x=pca_df[,1:5],grouping=k2$cluster,CV=TRUE)
ldak3<-lda(x=pca_df[,1:5],grouping=k3$cluster,CV=TRUE)
ldak4<-lda(x=pca_df[,1:5],grouping=k4$cluster,CV=TRUE)
ldak5<-lda(x=pca_df[,1:5],grouping=k5$cluster,CV=TRUE)
ldak6<-lda(x=pca_df[,1:5],grouping=k6$cluster,CV=TRUE)
ldak7<-lda(x=pca_df[,1:5],grouping=k7$cluster,CV=TRUE)
ldak8<-lda(x=pca_df[,1:5],grouping=k8$cluster,CV=TRUE)
ldak9<-lda(x=pca_df[,1:5],grouping=k9$cluster,CV=TRUE)
ldak10<-lda(x=pca_df[,1:5],grouping=k10$cluster,CV=TRUE)
ldak11<-lda(x=pca_df[,1:5],grouping=k11$cluster,CV=TRUE)

write.table(round(ldak2$posterior,5),file="ldak2.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak3$posterior,5),file="ldak3.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak4$posterior,5),file="ldak4.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak5$posterior,5),file="ldak5.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak6$posterior,5),file="ldak6.txt",quote=F,row.names=F,col.names=F)                                      
write.table(round(ldak7$posterior,5),file="ldak7.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak8$posterior,5),file="ldak8.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak9$posterior,5),file="ldak9.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak10$posterior,5),file="ldak10.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak11$posterior,5),file="ldak11.txt",quote=F,row.names=F,col.names=F)


In [204]:
%%R
######### create entropy header ####

Pop_ID_list <- paste(Pop_ID$Pop,Pop_ID$ID,sep='_')

Header <- data.frame(dims = NA,Pop_ID_list)

dim(pntest_mean)

df <- t(Header)
dims <- paste(dim(pntest_mean)[2],dim(pntest_mean)[1],sep = " ")

df[1,1] <- dims

write.table(df,'entropy_header.txt',sep = " ",na ="",
            quote = FALSE,row.names = FALSE,col.names = FALSE)

In [205]:
!head entropy_header.txt

194 6343                                                                                                                                                                                                 
BM_1 BM_3 BM_4 DT_2 DT_3 BM_1 BM_2 BM_3 BM_4 BM_5 BM_6 BM_7 BM_9 BM_10 BM_12 BM_13 BM_14 BM_15 BM_16 BM_17 BM_18 BM_19 BM_20 BM_21 BM_22 BM_23 BM_24 BM_25 BM_26 BM_27 BM_28 BM_29 BM_30 BM_31 BM_35 BM_36 BM_37 BM_38 BM_39 BM_40 DT_1 DT_2 DT_3 DT_4 DT_5 DT_6 DT_7 DT_8 DT_20 DT_21 DT_22 DT_23 DT_24 DT_25 DT_26 DT_27 DT_31 DT_32 DT_33 DT_34 DT_35 DT_36 DT_37 DT_38 DT_39 DT_40 DT_41 DT_42 DT_43 DT_44 DT_45 DT_46 LF_21 LF_22 LF_23 LF_24 LF_25 LF_26 LF_27 LF_28 LF_29 LF_2 LF_30 LF_31 LF_32 LF_33 LF_34 LF_35 LF_36 LF_37 LF_38 LF_39 LF_40 LF_41 LF_42 LF_43 LF_5 LF_99 BM_1 BM_2 BM_3 BM_4 BM_5 BM_6 BM_8 BM_9 BM_10 BM_11 BM_12 BM_13 BM_14 BM_15 BM_16 BM_17 BM_18 BM_19 BM_20 BM_21 BM_22 BM_23 BM_24 BM_25 BM_27 BM_28 BM_29 BM_30 BM_31 BM_32 BM_33 BM_34 BM_35 DT_1 DT_2 DT_3 DT_4 DT_5 DT_6 DT_7 DT_8 DT_

In [206]:
!cat entropy_header.txt mpgl.txt > entropy.mpgl

In [207]:
!head entropy.mpgl

194 6343                                                                                                                                                                                                 
BM_1 BM_3 BM_4 DT_2 DT_3 BM_1 BM_2 BM_3 BM_4 BM_5 BM_6 BM_7 BM_9 BM_10 BM_12 BM_13 BM_14 BM_15 BM_16 BM_17 BM_18 BM_19 BM_20 BM_21 BM_22 BM_23 BM_24 BM_25 BM_26 BM_27 BM_28 BM_29 BM_30 BM_31 BM_35 BM_36 BM_37 BM_38 BM_39 BM_40 DT_1 DT_2 DT_3 DT_4 DT_5 DT_6 DT_7 DT_8 DT_20 DT_21 DT_22 DT_23 DT_24 DT_25 DT_26 DT_27 DT_31 DT_32 DT_33 DT_34 DT_35 DT_36 DT_37 DT_38 DT_39 DT_40 DT_41 DT_42 DT_43 DT_44 DT_45 DT_46 LF_21 LF_22 LF_23 LF_24 LF_25 LF_26 LF_27 LF_28 LF_29 LF_2 LF_30 LF_31 LF_32 LF_33 LF_34 LF_35 LF_36 LF_37 LF_38 LF_39 LF_40 LF_41 LF_42 LF_43 LF_5 LF_99 BM_1 BM_2 BM_3 BM_4 BM_5 BM_6 BM_8 BM_9 BM_10 BM_11 BM_12 BM_13 BM_14 BM_15 BM_16 BM_17 BM_18 BM_19 BM_20 BM_21 BM_22 BM_23 BM_24 BM_25 BM_27 BM_28 BM_29 BM_30 BM_31 BM_32 BM_33 BM_34 BM_35 DT_1 DT_2 DT_3 DT_4 DT_5 DT_6 DT_7 DT_8 DT_

In [208]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy


In [209]:
!mkdir shdir

In [210]:
### select slurm options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
time = '4-00:00:00' #time limit 4
cpus = 4 #to prevent hammering one node
mem_cpu = 8000
email = 'tfaske@nevada.unr.edu'

#entropy settings
l = 60000
b = 10000
num_k = [2,3,4,5,6,7,8]
chains = 4

In [211]:
def write_entropy_sh(account,partition,time,cpus,mem_cpu,email,l,b,num_k,chains):
    for k in num_k:
        for c in range(1,chains+1):
            jobname = 'k%d_c%d' % (k,c)
            seed = random.randint(1000,9999) #default is clock so messes up if submitted same time 
            with open("shdir/run_entropy_%s.sh" % (jobname), "w") as o:
                o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name ent_%s
#SBATCH --output output_%s.txt
#SBATCH --mail-type=END
#SBATCH --mail-user=%s\n\n""" % (account,partition,time,cpus,mem_cpu,jobname,jobname,email))
        
                o.write("entropy -i ../entropy.mpgl -o ../entropy_%s.hdf5 -r %d -n 2 -l %d -b %d -t 10 -s 50 -e .01 -k %d -q ../ldak%d.txt -m 1 -w 0"
                        % (jobname,seed,l,b,k,k))
    

In [212]:
write_entropy_sh(account,partition,time,cpus,mem_cpu,email,l,b,num_k,chains)

In [213]:
entShells = !find $ent_dir -name 'run_entropy*.sh'
entShells = entShells.sort()
entShells

['/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k2_c1.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k2_c2.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k2_c3.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k2_c4.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k3_c1.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k3_c2.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k3_c3.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k3_c4.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k4_c1.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k4_c2.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k4_c3.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_entropy_k4_c4.sh',
 '/data/gpfs/assoc/denovo/tf

In [214]:
def write_sbatch_entShells(entShells):
         with open("shdir/sbatch_entropy.sh", "w") as o:
            o.write("""#!/usr/bin/env bash \n""")
            for sh in entShells:
                o.write("""sbatch %s\n""" % (sh))

In [215]:
write_sbatch_entShells(entShells)

# Run all entropy*sh
    cd /data/gpfs/home/tfaske/d/connie/ch2/entropy/shdir
    source activate entropy
    bash sbatch_entropy.sh

# Use estpost to get DIC, q, and gprob

In [216]:
np.set_printoptions(precision=8) # increases float print option
pd.set_option("precision", 8)

In [217]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy


In [218]:
hdf5_files = !find . -name '*hdf5'
hdf5_files = hdf5_files.sort()
hdf5_files

['./entropy_k2_c1.hdf5',
 './entropy_k2_c2.hdf5',
 './entropy_k2_c3.hdf5',
 './entropy_k2_c4.hdf5',
 './entropy_k3_c1.hdf5',
 './entropy_k3_c2.hdf5',
 './entropy_k3_c3.hdf5',
 './entropy_k3_c4.hdf5',
 './entropy_k4_c1.hdf5',
 './entropy_k4_c2.hdf5',
 './entropy_k4_c3.hdf5',
 './entropy_k4_c4.hdf5',
 './entropy_k5_c1.hdf5',
 './entropy_k5_c2.hdf5',
 './entropy_k5_c3.hdf5',
 './entropy_k5_c4.hdf5',
 './entropy_k6_c1.hdf5',
 './entropy_k6_c2.hdf5',
 './entropy_k6_c3.hdf5',
 './entropy_k6_c4.hdf5',
 './entropy_k7_c1.hdf5',
 './entropy_k7_c2.hdf5',
 './entropy_k7_c3.hdf5',
 './entropy_k7_c4.hdf5',
 './entropy_k8_c1.hdf5',
 './entropy_k8_c2.hdf5',
 './entropy_k8_c3.hdf5',
 './entropy_k8_c4.hdf5']

In [219]:
estpost = '/data/gpfs/home/tfaske/g/anaconda3/envs/entropy/bin/estpost.entropy'

In [220]:
#make DIC
for i in range(0,len(hdf5_files)):
    f = hdf5_files[i]
    k = f.split('_')[1] #set this 
    c = f.split('_')[2].split('.hdf5')[0]
    #print(k,c)
    dic = "DIC_%s_%s.txt" % (k,c)
    !$estpost $f -s 3 -p deviance > $dic

In [221]:
dic_files = !find . -name 'DIC*'
dic_files

['./DIC_k2_c4.txt',
 './DIC_k7_c2.txt',
 './DIC_k2_c1.txt',
 './DIC_k6_c1.txt',
 './DIC_k3_c2.txt',
 './DIC_k4_c4.txt',
 './DIC_k4_c2.txt',
 './DIC_k7_c1.txt',
 './DIC_k7_c4.txt',
 './DIC_k5_c2.txt',
 './DIC_k6_c2.txt',
 './DIC_k4_c3.txt',
 './DIC_k7_c3.txt',
 './DIC_k3_c1.txt',
 './DIC_k2_c2.txt',
 './DIC_k5_c4.txt',
 './DIC_k8_c4.txt',
 './DIC_k6_c3.txt',
 './DIC_k8_c3.txt',
 './DIC_k8_c1.txt',
 './DIC_k4_c1.txt',
 './DIC_k2_c3.txt',
 './DIC_k8_c2.txt',
 './DIC_k6_c4.txt',
 './DIC_k3_c4.txt',
 './DIC_k5_c1.txt',
 './DIC_k3_c3.txt',
 './DIC_k5_c3.txt']

In [222]:
for d in dic_files:
    !cat $d
    print('\n')

file = ./entropy_k2_c4.hdf5
parameter dimensions for deviance: samples = 5000, chains = 1
Model deviance: 2600471.09
Effective number of parameters: 5163022.60
Model DIC: 7763493.69


file = ./entropy_k7_c2.hdf5
parameter dimensions for deviance: samples = 5000, chains = 1
Model deviance: 2430625.28
Effective number of parameters: 20804812.24
Model DIC: 23235437.52


file = ./entropy_k2_c1.hdf5
parameter dimensions for deviance: samples = 5000, chains = 1
Model deviance: 2600440.97
Effective number of parameters: 5061892.05
Model DIC: 7662333.02


file = ./entropy_k6_c1.hdf5
parameter dimensions for deviance: samples = 5000, chains = 1
Model deviance: 2598877.62
Effective number of parameters: 54007070.06
Model DIC: 56605947.68


file = ./entropy_k3_c2.hdf5
parameter dimensions for deviance: samples = 5000, chains = 1
Model deviance: 2598817.39
Effective number of parameters: 29035478.94
Model DIC: 31634296.32


file = ./entropy_k4_c4.hdf5
parameter dimensions for deviance: samples = 5

In [223]:
dic_list = []
for d in dic_files:
    k = d.split('_k')[1].split('_')[0] #set this 
    c = d.split('_c')[1].split('.txt')[0]
    #print(k,c)
    
    dic = !grep 'DIC' $d
    dic = float(re.search('(\d+.\d+)',str(dic)).group(0))
    #print(dic)
    
    dic_list.append([k,dic,c])
dic_df = pd.DataFrame(dic_list,columns=['k','DIC','chain'])
dic_df.head()

,k,DIC,chain
0,2,7763493.69,4
1,7,23235437.52,2
2,2,7662333.02,1
3,6,56605947.68,1
4,3,31634296.32,2


In [224]:
dic_df.to_csv('dic_list.csv')

In [225]:
dic_sum = dic_df.groupby('k').describe().DIC

In [226]:
dic_sum.sort_values('mean')

,count,mean,std,min,25%,50%,75%,max
k,,,,,,,,
2,4.0,7.75904283e+06,1.07505472e+05,7662333.02,7.69253020e+06,7.73304481e+06,7.79955744e+06,7.90774868e+06
7,4.0,1.56167843e+08,9.43921143e+07,23235437.52,1.26688424e+08,1.80339450e+08,2.09818870e+08,2.40757036e+08
5,4.0,4.49283355e+08,4.33087349e+08,56484031.51,8.34191389e+07,4.56633293e+08,8.22497509e+08,8.27382800e+08
8,4.0,5.71312241e+08,3.68673795e+08,272083887.31,3.40153224e+08,4.58962373e+08,6.90121390e+08,1.09524033e+09
3,4.0,7.38152283e+08,4.89003864e+08,31634296.32,6.24771907e+08,8.89044477e+08,1.00242485e+09,1.14288588e+09
4,4.0,7.70237112e+08,1.15648985e+09,7594790.78,1.14112311e+07,3.10342626e+08,1.06916851e+09,2.45266841e+09
6,4.0,1.04022615e+09,1.07821246e+09,56605947.68,3.39476892e+08,8.01321841e+08,1.50207110e+09,2.50165499e+09


In [227]:
dic_sum.to_csv('dic_sum.csv')

### Get gprob and q for each k, and mcmc metrics

In [228]:
# ancestry coeffecients 
!$estpost *k2*.hdf5 -p q -s 0 -o q2.txt

!$estpost *k3*.hdf5 -p q -s 0 -o q3.txt

!$estpost *k4*.hdf5 -p q -s 0 -o q4.txt

!$estpost *k5*.hdf5 -p q -s 0 -o q5.txt

!$estpost *k6*.hdf5 -p q -s 0 -o q6.txt

!$estpost *k7*.hdf5 -p q -s 0 -o q7.txt

!$estpost *k8*.hdf5 -p q -s 0 -o q8.txt


file = entropy_k2_c1.hdf5
file = entropy_k2_c2.hdf5
file = entropy_k2_c3.hdf5
file = entropy_k2_c4.hdf5
parameter dimensions for q: ind = 194, populations = 2, samples = 5000, chains = 4
file = entropy_k3_c1.hdf5
file = entropy_k3_c2.hdf5
file = entropy_k3_c3.hdf5
file = entropy_k3_c4.hdf5
parameter dimensions for q: ind = 194, populations = 3, samples = 5000, chains = 4
file = entropy_k4_c1.hdf5
file = entropy_k4_c2.hdf5
file = entropy_k4_c3.hdf5
file = entropy_k4_c4.hdf5
parameter dimensions for q: ind = 194, populations = 4, samples = 5000, chains = 4
file = entropy_k5_c1.hdf5
file = entropy_k5_c2.hdf5
file = entropy_k5_c3.hdf5
file = entropy_k5_c4.hdf5
parameter dimensions for q: ind = 194, populations = 5, samples = 5000, chains = 4
file = entropy_k6_c1.hdf5
file = entropy_k6_c2.hdf5
file = entropy_k6_c3.hdf5
file = entropy_k6_c4.hdf5
parameter dimensions for q: ind = 194, populations = 6, samples = 5000, chains = 4
file = entropy_k7_c1.hdf5
file = entropy_k7_c2.hdf5
file = entrop

In [229]:
#MCMC diagnostics
!$estpost *k2*.hdf5 -p q -s 4 -o MCMC_k2.txt

!$estpost *k3*.hdf5 -p q -s 4 -o MCMC_k3.txt

!$estpost *k4*.hdf5 -p q -s 4 -o MCMC_k4.txt

!$estpost *k5*.hdf5 -p q -s 4 -o MCMC_k5.txt

!$estpost *k6*.hdf5 -p q -s 4 -o MCMC_k6.txt

!$estpost *k7*.hdf5 -p q -s 4 -o MCMC_k7.txt

!$estpost *k8*.hdf5 -p q -s 4 -o MCMC_k8.txt

file = entropy_k2_c1.hdf5
file = entropy_k2_c2.hdf5
file = entropy_k2_c3.hdf5
file = entropy_k2_c4.hdf5
parameter dimensions for q: ind = 194, populations = 2, samples = 5000, chains = 4
file = entropy_k3_c1.hdf5
file = entropy_k3_c2.hdf5
file = entropy_k3_c3.hdf5
file = entropy_k3_c4.hdf5
parameter dimensions for q: ind = 194, populations = 3, samples = 5000, chains = 4
file = entropy_k4_c1.hdf5
file = entropy_k4_c2.hdf5
file = entropy_k4_c3.hdf5
file = entropy_k4_c4.hdf5
parameter dimensions for q: ind = 194, populations = 4, samples = 5000, chains = 4
file = entropy_k5_c1.hdf5
file = entropy_k5_c2.hdf5
file = entropy_k5_c3.hdf5
file = entropy_k5_c4.hdf5
parameter dimensions for q: ind = 194, populations = 5, samples = 5000, chains = 4
file = entropy_k6_c1.hdf5
file = entropy_k6_c2.hdf5
file = entropy_k6_c3.hdf5
file = entropy_k6_c4.hdf5
parameter dimensions for q: ind = 194, populations = 6, samples = 5000, chains = 4
file = entropy_k7_c1.hdf5
file = entropy_k7_c2.hdf5
file = entrop

### make slurm script to make gprob file

    use all k and chains to make gprob

In [230]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy


In [231]:
### select slurm options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
time = '1-00:00:00' #time limit 4
cpus = 2 #to prevent hammering one node
mem_cpu = 6000
email = 'tfaske@nevada.unr.edu'

### select k 
num_k = [2,3,4,6,7,8]

In [232]:
def write_gprob_k_sh(account,partition,time,cpus,mem_cpu,email,estpost,num_k):
    for k in num_k:
        with open("shdir/run_gprob_k%d.sh" % (k), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name gprob_k%d
#SBATCH --output output_gprob_k%d.txt
#SBATCH --mail-type=END
#SBATCH --mail-user=%s\n\n""" % (account,partition,time,cpus,mem_cpu,k,k,email))
        
            o.write("%s ../*k%d*.hdf5 -p gprob -s 0 -o ../gprob%d.txt" % (estpost,k,k))

In [233]:
write_gprob_k_sh(account,partition,time,cpus,mem_cpu,email,estpost,num_k)

In [234]:
gprobShells = !find $ent_dir -name 'run_gprob*.sh'
gprobShells = gprobShells.sort()
gprobShells

['/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_gprob_k2.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_gprob_k3.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_gprob_k4.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_gprob_k6.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_gprob_k7.sh',
 '/data/gpfs/assoc/denovo/tfaske/connie/ch2/entropy/shdir/run_gprob_k8.sh']

In [235]:
def write_sbatch_gprobShells(gprobShells):
    with open("shdir/sbatch_gprob.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n""")
        for sh in gprobShells:
            o.write("""sbatch %s\n""" % (sh))

In [236]:
write_sbatch_gprobShells(gprobShells)

### gprob all 

In [237]:
hdf5_files = []
for k in num_k:
    f = '../entropy_k' + str(k) + '_c*hdf5'
    hdf5_files.append(f)
hdf5_files

['../entropy_k2_c*hdf5',
 '../entropy_k3_c*hdf5',
 '../entropy_k4_c*hdf5',
 '../entropy_k6_c*hdf5',
 '../entropy_k7_c*hdf5',
 '../entropy_k8_c*hdf5']

In [238]:
gprob_cmd = estpost + ' ' + ' '.join(hdf5_files) + ' -p gprob -s 0 -o ../gprobAll.txt'
gprob_cmd

'/data/gpfs/home/tfaske/g/anaconda3/envs/entropy/bin/estpost.entropy ../entropy_k2_c*hdf5 ../entropy_k3_c*hdf5 ../entropy_k4_c*hdf5 ../entropy_k6_c*hdf5 ../entropy_k7_c*hdf5 ../entropy_k8_c*hdf5 -p gprob -s 0 -o ../gprobAll.txt'

In [239]:
### select slurm options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
time = '1-00:00:00' #time limit 4
cpus = 2 #to prevent hammering one node
mem_cpu = 100000
email = 'tfaske@nevada.unr.edu'

In [240]:
def write_gprob_all_sh(account,partition,time,cpus,mem_cpu,email,estpost,gprob_cmd):
        with open("shdir/run_gprobAll.sh", "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name gprobAll
#SBATCH --output output_gprobAll.txt
#SBATCH --mail-type=END
#SBATCH --mail-user=%s

%s""" % (account,partition,time,cpus,mem_cpu,email,gprob_cmd))

In [241]:
write_gprob_all_sh(account,partition,time,cpus,mem_cpu,email,estpost,gprob_cmd)

## Run gprob sh 
    cd /data/gpfs/home/tfaske/d/milkweed/entropy
    source activate entropy
    sbatch run_gprob.sh

In [ ]:
#gprobs
#!$estpost *k2*.hdf5 -p gprob -s 0 -o gprob2.txt

#!$estpost *k3*.hdf5 -p gprob -s 0 -o gprob3.txt

#!$estpost *k4*.hdf5 -p gprob -s 0 -o gprob4.txt

#!$estpost *k5*.hdf5 -p gprob -s 0 -o gprob5.txt